In [2]:
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from statsmodels.nonparametric.kernel_density import KDEMultivariate
# from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupShuffleSplit
#from sklearn.model_selection import train_test_split

import seaborn as sns
import numpy as np
import re
import os


In [3]:
%load_ext autoreload
%autoreload 2
    
from helpers.utils import _qq_stats_text
from helpers.utils import scree
from helpers.utils import learn_kde
from helpers.utils import score_with_kdes
from helpers.utils import reshape_to_cmat, _site_slice
import helpers.utils

In [4]:
plot_dir = "/Users/gc3045/git/laml2-experiments/real_data/PEtracer/plots/"
out_dir = "/Users/gc3045/git/laml2-experiments/real_data/PEtracer/inputs/"

In [5]:
training_data = out_dir + "petracer_full_training.csv"
barcoded_fname = "/Users/gc3045/git/laml2-experiments/real_data/PEtracer/PEtracer raw data/barcoded_tracing_alleles.csv"
barcoded_unfiltered_fname = "/Users/gc3045/git/laml2-experiments/real_data/PEtracer/PEtracer raw data/240517_colony/decoded_spots.csv"

### Read inputs

In [20]:
barcoded_df

,cellBC,RNF2,HEK3,EMX1,UMI,readCount,sample,reads_per_umi,clone,whitelist
intID,,,,,,,,,,
intID1012,pool2-AAACCCAAGTCAGCGA-1,TATAT,CTTTG,CCTTT,39,334,pool2,8.564103,4,True
intID1012,pool2-AAACCCAGTGATACTC-1,NaN,GCAAG,AGTAC,69,555,pool2,8.043478,4,True
intID1012,pool2-AAACGAAAGCGTGTTT-1,NaN,CTCTC,CCTTT,92,671,pool2,7.293478,4,True
intID1012,pool2-AAACGAACAGCCGGTT-1,ACTCC,CTTTG,CA,44,355,pool2,8.068182,4,True
intID1012,pool2-AAACGCTAGCATGATA-1,TATAT,GCGCC,CCTTT,45,324,pool2,7.200000,4,True
...,...,...,...,...,...,...,...,...,...,...
intID991,pool3-TTTGGAGGTATGACAA-1,ACTTA,ATTTA,AGTAC,31,427,pool3,13.774194,3,True
intID991,pool3-TTTGGAGGTCGCAACC-1,ACTTA,NaN,AGTAC,91,1447,pool3,15.901099,3,True
intID991,pool3-TTTGGTTCACCCAACG-1,ACTTA,NaN,AGTAC,107,1654,pool3,15.457944,3,True


In [6]:
barcoded_df = pd.read_csv(barcoded_fname,index_col=0)
barcoded_unfiltered_df = pd.read_csv(barcoded_unfiltered_fname, index_col=0)

In [7]:
barcoded_unfiltered_df.columns

Index(['spot', 'x', 'y', 'z', 'global_x', 'global_y', 'global_z', 'r1', 'r2',
       'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13',
       'r14', 'r15', 'r16', 'r17', 'r18', 'r19', 'r20', 'r21', 'r25', 'r26',
       'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33', 'r34', 'r35', 'r36',
       'r37', 'r38', 'r39', 'r40', 'r41', 'r42', 'r43', 'r44', 'r45', 'r46',
       'r47', 'r48', 'r49', 'r50', 'r51', 'intBC', 'intBC_dist',
       'intBC_intensity', 'intBC_snr', 'HEK3', 'HEK3_prob', 'HEK3_intensity',
       'HEK3_snr', 'EMX1', 'EMX1_prob', 'EMX1_intensity', 'EMX1_snr', 'RNF2',
       'RNF2_prob', 'RNF2_intensity', 'RNF2_snr', 'cell', 'cell_dist'],
      dtype='object')

In [8]:
barcoded_unfiltered_df.shape

(662443, 73)

In [15]:
df = barcoded_df.copy()

In [16]:
df

,cellBC,RNF2,HEK3,EMX1,UMI,readCount,sample,reads_per_umi,clone,whitelist
intID,,,,,,,,,,
intID1012,pool2-AAACCCAAGTCAGCGA-1,TATAT,CTTTG,CCTTT,39,334,pool2,8.564103,4,True
intID1012,pool2-AAACCCAGTGATACTC-1,NaN,GCAAG,AGTAC,69,555,pool2,8.043478,4,True
intID1012,pool2-AAACGAAAGCGTGTTT-1,NaN,CTCTC,CCTTT,92,671,pool2,7.293478,4,True
intID1012,pool2-AAACGAACAGCCGGTT-1,ACTCC,CTTTG,CA,44,355,pool2,8.068182,4,True
intID1012,pool2-AAACGCTAGCATGATA-1,TATAT,GCGCC,CCTTT,45,324,pool2,7.200000,4,True
...,...,...,...,...,...,...,...,...,...,...
intID991,pool3-TTTGGAGGTATGACAA-1,ACTTA,ATTTA,AGTAC,31,427,pool3,13.774194,3,True
intID991,pool3-TTTGGAGGTCGCAACC-1,ACTTA,NaN,AGTAC,91,1447,pool3,15.901099,3,True
intID991,pool3-TTTGGTTCACCCAACG-1,ACTTA,NaN,AGTAC,107,1654,pool3,15.457944,3,True


In [17]:
mask = df['clone'] == 4
df = df.loc[mask]

In [19]:
df['cellBC'].nunique()

3108

In [7]:
df_train = pd.read_csv(training_data)

,cellBC,RNF2,HEK3,EMX1,UMI,readCount,sample,reads_per_umi,clone,whitelist
intID,,,,,,,,,,
intID1012,pool2-AAACCCAAGTCAGCGA-1,TATAT,CTTTG,CCTTT,39,334,pool2,8.564103,4,True
intID1012,pool2-AAACCCAGTGATACTC-1,NaN,GCAAG,AGTAC,69,555,pool2,8.043478,4,True
intID1012,pool2-AAACGAAAGCGTGTTT-1,NaN,CTCTC,CCTTT,92,671,pool2,7.293478,4,True
intID1012,pool2-AAACGAACAGCCGGTT-1,ACTCC,CTTTG,CA,44,355,pool2,8.068182,4,True
intID1012,pool2-AAACGCTAGCATGATA-1,TATAT,GCGCC,CCTTT,45,324,pool2,7.200000,4,True
...,...,...,...,...,...,...,...,...,...,...
intID991,pool3-TTTGGAGGTATGACAA-1,ACTTA,ATTTA,AGTAC,31,427,pool3,13.774194,3,True
intID991,pool3-TTTGGAGGTCGCAACC-1,ACTTA,NaN,AGTAC,91,1447,pool3,15.901099,3,True
intID991,pool3-TTTGGTTCACCCAACG-1,ACTTA,NaN,AGTAC,107,1654,pool3,15.457944,3,True


In [9]:
df.columns

Index(['cellBC', 'RNF2', 'HEK3', 'EMX1', 'UMI', 'readCount', 'sample',
       'reads_per_umi', 'clone', 'whitelist'],
      dtype='object')

Let's subset down to Tumor 1 for this mouse first.

In [10]:
df_test = df[df['clone'] == 4]

In [11]:
df_test.shape

(33430, 10)

In [12]:
df_test['cellBC'].nunique()

3108

In [13]:
df['cellBC'].nunique()

24806

In [43]:
df_test['sample']

0         M1-1
1         M1-1
2         M1-1
3         M1-1
4         M1-1
          ... 
584867    M1-4
584868    M1-4
584871    M1-4
584872    M1-4
584882    M1-4
Name: sample, Length: 349371, dtype: object

In [13]:
id_cols    = ["cellBC", "intID", "clone", "x", "y", "z"]

pieces = [
    _site_slice(df_test, "HEK3", 0, id_cols),
    _site_slice(df_test, "EMX1", 1, id_cols),
    _site_slice(df_test, "RNF2", 2, id_cols),
]

test_df_long = pd.concat(pieces, axis=0, ignore_index=True)

Note this helper function is specific to PEtracer. Assumes the features are r25-r51.
['r25', 'r26', 'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33'] [True, True, True, True, True, True, True, True, True]
Note this helper function is specific to PEtracer. Assumes the features are r25-r51.
['r34', 'r35', 'r36', 'r37', 'r38', 'r39', 'r40', 'r41', 'r42'] [True, True, True, True, True, True, True, True, True]
Note this helper function is specific to PEtracer. Assumes the features are r25-r51.
['r43', 'r44', 'r45', 'r46', 'r47', 'r48', 'r49', 'r50', 'r51'] [True, True, True, True, True, True, True, True, True]


In [14]:
test_df_long

,cellBC,intID,clone,x,y,z,target_site,pet_state,pet_prob,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8
0,M1-1-1000,intID120,1,1943,1331,41,HEK3,unedited,0.976,284.778,358.135,222.089,210.903,279.671,302.224,246.649,303.754,249.564
1,M1-1-10028,intID120,1,180,1605,36,HEK3,unedited,0.996,469.467,600.516,286.197,706.583,634.698,219.799,375.335,388.432,326.265
2,M1-1-10035,intID120,1,1977,1568,30,HEK3,unedited,0.837,266.905,274.233,274.749,260.947,245.489,290.776,260.947,424.168,271.315
3,M1-1-1005,intID120,1,1784,1285,37,HEK3,unedited,1.000,171.582,181.786,278.183,270.480,299.870,259.867,187.072,183.340,201.483
4,M1-1-10053,intID120,1,1244,1746,20,HEK3,unedited,0.715,219.243,189.555,306.803,244.266,336.382,372.056,206.136,295.985,233.537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048108,M1-4-9911,intID760,1,2052,884,48,RNF2,TCCAA,0.901,415.062,467.533,301.557,571.307,2551.690,657.842,348.271,713.936,595.297
1048109,M1-4-9934,intID760,1,422,2265,32,RNF2,unedited,0.998,279.094,168.217,234.545,121.656,303.265,261.350,237.349,340.383,2234.876
1048110,M1-4-9944,intID760,1,1509,836,33,RNF2,ACTCC,0.844,242.120,227.449,280.337,245.698,247.982,854.413,273.130,256.669,284.804
1048111,M1-4-9945,intID760,1,1933,846,37,RNF2,GTTCA,0.995,189.641,211.654,1941.136,288.635,247.982,474.674,327.995,304.055,340.648


In [15]:
# what's the size of this? 
test_df_long['cellBC'].nunique(), test_df_long['intID'].nunique() * 3

(53118, 36)

In [16]:
unique_cassettes = test_df_long['intID'].unique()
cassette_mapping = {id_: i for i, id_ in enumerate(unique_cassettes)}
test_df_long["cassette_idx"] = test_df_long["intID"].map(cassette_mapping)

site_mapping = {'RNF2': 0, 'HEK3': 1, 'EMX1': 2}
test_df_long['target_idx'] = test_df_long['target_site'].map(site_mapping) + len(site_mapping) * test_df_long["cassette_idx"]

In [22]:
tbl = test_df_long[['target_idx', 'cellBC', 'pet_state']]
tbl.columns = ['target_site', 'cell_name', 'pred_label']
baseMem_inputs_argmax = (tbl.pivot_table(columns='target_site',
                                         index='cell_name',
                                         values='pred_label',
                                         aggfunc='first')).sort_index().sort_index(axis=1).fillna('missing')

In [23]:
baseMem_inputs_argmax

target_site,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
cell_name,,,,,,,,,,,,,,,,,,,,,
M1-1-1000,unedited,unedited,unedited,missing,missing,missing,TATAT,GCAAG,CCCTA,ACAGT,...,missing,unedited,CTCTC,GGACA,TCCAA,unedited,ATCAA,missing,missing,missing
M1-1-10006,missing,missing,missing,unedited,AATCG,unedited,missing,missing,missing,missing,...,missing,unedited,GCGCC,AGTAC,missing,missing,missing,missing,missing,missing
M1-1-10009,missing,missing,missing,GTTCA,ATCAA,unedited,missing,missing,missing,ACAGT,...,CCGAT,missing,missing,missing,missing,missing,missing,ACAGT,CTCTC,CCCTA
M1-1-1001,missing,missing,missing,missing,missing,missing,unedited,GCAAG,ACAAT,missing,...,missing,TTCCT,unedited,unedited,TGCCA,unedited,ACAAT,missing,missing,missing
M1-1-10028,unedited,unedited,ATCAA,unedited,AATCG,unedited,GTTCA,GCAAG,unedited,ACAGT,...,CCGAT,unedited,GCGCC,AGTAC,ACTCC,unedited,AGTAC,ACTCC,GCGCC,unedited
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M1-4-9944,missing,missing,missing,unedited,AATCG,GGACA,missing,missing,missing,ACAGT,...,CCGAT,missing,missing,missing,TCCAA,ATCAA,CCGAT,ACTCC,CTTTG,GGACA
M1-4-9945,ACTTA,unedited,unedited,missing,missing,missing,TCCAA,GCAAG,unedited,ACAGT,...,missing,ACTTA,ATTTA,AGTAC,TCCAA,GATAG,ACAAT,GTTCA,CTTTG,GGACA
M1-4-9949,unedited,unedited,GGACA,unedited,AATCG,unedited,TCCAA,GCAAG,unedited,ACAGT,...,CCGAT,TTCCT,GCAAG,unedited,unedited,unedited,CCTTT,missing,missing,missing


In [24]:
df_dedup = baseMem_inputs_argmax.drop_duplicates(keep='first')
df_dedup.shape

(52961, 36)

### Learn 27 KDEs on all the training data 

In [27]:
barcoded_df

,cellBC,RNF2,HEK3,EMX1,UMI,readCount,sample,reads_per_umi,clone,whitelist
intID,,,,,,,,,,
intID1012,pool2-AAACCCAAGTCAGCGA-1,TATAT,CTTTG,CCTTT,39,334,pool2,8.564103,4,True
intID1012,pool2-AAACCCAGTGATACTC-1,NaN,GCAAG,AGTAC,69,555,pool2,8.043478,4,True
intID1012,pool2-AAACGAAAGCGTGTTT-1,NaN,CTCTC,CCTTT,92,671,pool2,7.293478,4,True
intID1012,pool2-AAACGAACAGCCGGTT-1,ACTCC,CTTTG,CA,44,355,pool2,8.068182,4,True
intID1012,pool2-AAACGCTAGCATGATA-1,TATAT,GCGCC,CCTTT,45,324,pool2,7.200000,4,True
...,...,...,...,...,...,...,...,...,...,...
intID991,pool3-TTTGGAGGTATGACAA-1,ACTTA,ATTTA,AGTAC,31,427,pool3,13.774194,3,True
intID991,pool3-TTTGGAGGTCGCAACC-1,ACTTA,NaN,AGTAC,91,1447,pool3,15.901099,3,True
intID991,pool3-TTTGGTTCACCCAACG-1,ACTTA,NaN,AGTAC,107,1654,pool3,15.457944,3,True


In [29]:
df_train

,cellBC,intID,clone,x,y,z,target_site,pet_state,seq_state,brightest_state,pet_prob,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8
0,4T1_preedited-1057,intID1011,2,504,123,20,HEK3,CTCTC,CTCTC,CTCTC,0.999968,217.510,217.293,179.428,137.317,266.385,203.037,2130.065,343.644,168.804
1,4T1_preedited-1120,intID1011,2,1928,1712,26,HEK3,CTCTC,CTCTC,CTCTC,0.998348,381.193,1051.053,397.811,592.112,1058.296,211.300,3782.266,1129.117,207.759
2,4T1_preedited-1143,intID1011,2,1951,952,31,HEK3,CTCTC,CTCTC,CTCTC,0.999994,447.105,514.259,181.789,366.912,617.272,203.037,5035.697,709.823,158.180
3,4T1_preedited-1153,intID1011,2,2189,647,23,HEK3,CTCTC,CTCTC,CTCTC,0.999737,262.551,402.394,194.774,206.525,537.598,206.578,2346.477,414.466,212.481
4,4T1_preedited-1162,intID1011,2,1944,1198,6,HEK3,CTCTC,CTCTC,CTCTC,0.999808,1261.121,1508.173,310.458,1648.905,1928.272,246.714,9536.407,1950.001,322.262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131800,4T1_preedited-8630,intID974,2,378,1209,22,RNF2,TTCCT,TTCCT,TTCCT,0.999471,1132.592,273.628,125.128,141.711,148.886,197.135,187.850,159.348,181.789
131801,4T1_preedited-8638,intID974,2,84,519,31,RNF2,TTCCT,TTCCT,TTCCT,0.999980,4390.856,626.125,216.022,377.897,534.379,750.765,641.546,424.928,211.300
131802,4T1_preedited-8639,intID974,2,323,533,25,RNF2,TTCCT,TTCCT,TTCCT,0.991395,1024.936,212.464,147.556,175.766,169.810,514.676,176.865,141.643,174.706
131803,4T1_preedited-8644,intID974,2,431,423,36,RNF2,TTCCT,TTCCT,TTCCT,0.999988,2760.626,358.131,173.526,252.664,181.077,445.029,785.455,276.042,158.180


In [30]:
test_df_long

,cellBC,intID,clone,x,y,z,target_site,pet_state,pet_prob,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,cassette_idx,target_idx
0,M1-1-1000,intID120,1,1943,1331,41,HEK3,unedited,0.976,284.778,358.135,222.089,210.903,279.671,302.224,246.649,303.754,249.564,0,1
1,M1-1-10028,intID120,1,180,1605,36,HEK3,unedited,0.996,469.467,600.516,286.197,706.583,634.698,219.799,375.335,388.432,326.265,0,1
2,M1-1-10035,intID120,1,1977,1568,30,HEK3,unedited,0.837,266.905,274.233,274.749,260.947,245.489,290.776,260.947,424.168,271.315,0,1
3,M1-1-1005,intID120,1,1784,1285,37,HEK3,unedited,1.000,171.582,181.786,278.183,270.480,299.870,259.867,187.072,183.340,201.483,0,1
4,M1-1-10053,intID120,1,1244,1746,20,HEK3,unedited,0.715,219.243,189.555,306.803,244.266,336.382,372.056,206.136,295.985,233.537,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048108,M1-4-9911,intID760,1,2052,884,48,RNF2,TCCAA,0.901,415.062,467.533,301.557,571.307,2551.690,657.842,348.271,713.936,595.297,11,33
1048109,M1-4-9934,intID760,1,422,2265,32,RNF2,unedited,0.998,279.094,168.217,234.545,121.656,303.265,261.350,237.349,340.383,2234.876,11,33
1048110,M1-4-9944,intID760,1,1509,836,33,RNF2,ACTCC,0.844,242.120,227.449,280.337,245.698,247.982,854.413,273.130,256.669,284.804,11,33
1048111,M1-4-9945,intID760,1,1933,846,37,RNF2,GTTCA,0.995,189.641,211.654,1941.136,288.635,247.982,474.674,327.995,304.055,340.648,11,33


### Score the mouse tumor 1